<a href="https://colab.research.google.com/github/Atsaya-Poovasu/big-data-analytics/blob/main/bd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download and Install Apache Spark (with Findspark)
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

# Change Java Version Frim JDK-11 to JDK-8
#!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

# Set Environment Variables
#os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

tar: spark-2.4.4-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

tar: spark-2.4.4-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [31]:
!pip install pyspark


In [34]:
import findspark
#findspark.init()

# Start Apache Spark Session & Context
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext(appName='sd702-RecoSys-Models')
sqlContext = SQLContext(sc)

print('Master : ', sc.master)
print('Cores  : ', sc.defaultParallelism)

ValueError: ignored

In [35]:
sqlContext.sparkSession.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
!pip install koalas

In [37]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import random
from pprint import pprint
from matplotlib.lines import Line2D

# Downloaded Packages (not available by Default)
#import databricks.koalas

# PySpark Utilities
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.mllib.evaluation import RegressionMetrics, RankingMetrics

# Random Seed
SEED = 1492

# Set-up
plt.style.use('seaborn')

In [38]:
DATA_PATH = "drive/My Drive/BIG DATA/"
RESULTS_PATH = "drive/My Drive//BIG DATA/"

In [40]:
class MovieLensDatasets(object):
    """
    Class for loading and preprocesing MovieLens Dataset
    """
    def __init__(self, ratings, movies, links, personalRatings,
                 debug=True, debugLimit=10000):
    # Load Existing Data
      if debug:
          debugLimit = debugLimit
          ratings = ratings.limit(debugLimit)
      else:
          ratings = ratings      

      self.ratings = ratings
      self.movies = movies
      self.links = links
      self.personalRatings = personalRatings

      # Create New DataFrame
      users = ratings.select('userId').distinct()
      self.users = users

    def preprocessing(self):
    # Preprocess MovieLens Ratings
      self.ratings = self.ratings.withColumn('rating',
            F.col('rating').cast('double')).drop('timestamp') \
            .withColumn('userId', F.col('userId').cast('int')) \
            .withColumn('movieId', F.col('movieId').cast('int'))

      # Preprocess Personal IMDb Ratings To MovieLens Ratings
      self.personalRatings = self.personalRatings.select(['Const',
                                                      'Your Rating']) \
            .withColumnRenamed('Const', 'imdbId') \
            .withColumnRenamed('Your Rating', 'personalRating')

      self.personalRatings = self.personalRatings \
          .withColumn('personalRating', F.col('personalRating').cast('double')*0.5) \
          .withColumn('imdbId', F.expr("substr(imdbId, 3)"))

      self.personalRatings = self.personalRatings.join(
          self.links.select('movieId', 'imdbId'), ['imdbId'], how='inner')

      # Insert IMDb Ratings into MovieLens Ratings Dataset
      self.personalRatings = self.personalRatings \
                          .withColumn('userId', F.lit('0'))
      self.personalRatings = self.personalRatings \
                          .select(['userId', 'movieId', 'personalRating']) \
                          .toDF('userId', 'movieId', 'rating')
      self.ratings = self.ratings.union(self.personalRatings)

      # Binarize MovieLens Ratings (if rating >= 3.0, then 1.0, else 0.0)
      udf_scale_ratings = F.udf(lambda x: x - 2.5, DoubleType())
      udf_binary_ratings = F.udf(lambda x: 1.0 if x > 0.0 else 0.0, DoubleType())

      self.ratings = self.ratings \
      .withColumn('ratingsScaled', udf_scale_ratings(F.col('rating'))) \
      .withColumn('ratingsBinary', udf_binary_ratings(F.col('ratingsScaled')))

    def get_ratings(self):
        return self.ratings

    def get_movies(self):
        return self.movies

    # Displaying Null Values
    def spark_df_display_null_values(sparkDf):
        print('NaN values ?')
        sparkDf.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in sparkDf.columns]).show()

        print('Null values ?')
        sparkDf.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in sparkDf.columns]).show()

In [41]:
%%time
# Set to True if only testing
# For testing: use debug=True option (subset of all ratings)
debug = False

# Load Datasets
movies = sqlContext \
            .read.format('csv').option("header", "true") \
            .load(DATA_PATH + "movies.csv")

links = sqlContext \
            .read.format('csv').option("header", "true") \
            .load(DATA_PATH + "links.csv")

ratings = sqlContext.read \
            .format('csv').option("header", "true") \
            .load(DATA_PATH + "ratings.csv")

personalRatings = sqlContext \
            .read.format('csv').option("header", "true") \
            .load(DATA_PATH + "Personal_IMBD_Ratings.csv")


# Data Preprocessing
movieLensDatasets = MovieLensDatasets(ratings=ratings, movies=movies,
                                      links=links,
                                      personalRatings=personalRatings,
                                      debug=debug)

movieLensDatasets.preprocessing()
dfRatings = movieLensDatasets.get_ratings() 
dfMovies = movieLensDatasets.get_movies()

AnalysisException: ignored